## Teammates:
    Name: Raneem Ahmed
    ID: 49-9975
    Tutorial_Number: T04
    
    Name: Jasmine Yasser
    ID: 49-5873
    Tutorial_Number: T02
    
#### ---- We submitted the assignment twice and this is the Most Recent Version :)

## Import the needed libraries

In [1]:
from Crypto.Cipher import PKCS1_OAEP
from Crypto.PublicKey import RSA
import secrets
import random
from random import randint
from hashlib import sha512
import hashlib

# 1) TLS Simulation using RSA 

# Step 1
Client sends the server a 32 byte nonce along with a flag

In [2]:
#Generate Flag--> flag=1 use RSA, flag=0 use DiffieHellman
def randomFlag():
    flag= random.choice(range(0, 2))
    return flag
randomFlag()

1

In [3]:
#Generate client nonce -->Rb
def clientRb():
    client_nonce= bytes([random.randrange(0, 256) for _ in range(0, 32)])
    client_nonce= secrets.token_bytes()
    return client_nonce
client_nonce= clientRb()
client_nonce

b'G\xa7m\x82\x9c\xb7_@[\xebM\x7f\x93\x04D\t%\xc6\xf6J\xf2\xf8\xd8/+\\3\x8d3\xe7\xb1`'

In [4]:
#Generate length of client nonce
len(client_nonce)

32

# Step 2
Server sends the client a 32 byte nonce along with a flag

In [5]:
#Generate server nonce --> Rs
def serverRs():
    server_nonce= bytes([random.randrange(0, 256) for _ in range(0, 32)])
    server_nonce= secrets.token_bytes()
    return server_nonce
server_nonce= serverRs()
server_nonce

b'e\x99\x98\xe3\x99#q\x1b:\xaf\x1bU7\x9f\xee\xf9\xd1\xd6c&\xc2\x00\xd8t\xcf\xa2\xa1M\x9d\xe4\x8cR'

In [6]:
#Generate length of server nonce
len(server_nonce)

32

# Step 3
Server randomly generates a pair of RSA public/private keys and sends to the client {public key, hash(public key)} as a certificate 

In [7]:
rand= RSA.generate(2048)
PR= rand.exportKey("PEM")
PU= rand.publickey().exportKey("PEM")
fd= open("PR.pem", "wb")
fd.write(PR)
fd.close()
fd= open("PU.pem", "wb")
fd.write(PR)
fd.close()

In [8]:
PU

b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAmgmd0oB6C3J8R6max+5W\nTNjAa2DJjQyyBYUbhyb1unSrB/OuQTg+K7ggn5WWrz9nZ9uIg+Lk0TT93DOvxUvR\nXSSI4qYdu15u8p7BIznRUdXXD6cFLV5nwBRviuIrdBM+YG/EazdSGrF02R4SxNSx\nioXgMUsisVjIR56L80gI8y1NjLGJxXRMGAnsxzexY0absQKU2jdYJBd36DTxsYMN\nSd9/g0NqUUhjIGKJj1PCKWt3ygHx9n7LPb6rB9AIRj/82uvv9CDbah3EIZmw7mCi\nzIHWdjrcfKtCvRWBl3eJKwNi/C93f0HAbPozJEXTGbfC58SNf7ph7OlMrI1JUOs0\noQIDAQAB\n-----END PUBLIC KEY-----'

In [9]:
type(PU)

bytes

In [10]:
#Generate server PU Hash
def hashFunction(message):
    hash= int.from_bytes(sha512(message).digest(), byteorder= 'big')
    return(hex(hash))

In [11]:
server_PUhash= hashFunction(PU)
server_PUhash

'0x7c1da08ac96d6a08070ac1f464946802f77ae441fd756ab788bb46210ae4bbdae808bd4362fc8af0e41015fb281233730ba817172801780d4de1d91541ff695f'

# Step 4

In [12]:
#Client rehashes server's PU and compares both hashes
hashTester= hashFunction(PU)
if(hashTester== server_PUhash):
    z= 'Matching'
else:
    z= 'Not Matching'
z

'Matching'

# Step 5
Client exchanges the pre-master secret (PS)

In [13]:
#Generate Premaster Secret
PS= secrets.token_bytes()
PS

b'\xfb_G\xcc\xe6\x1cy\x04\x1f\x84Xz\x14\xd8]\xf7uX-{\xa6\xc0\x96\x9e*Y\xfd_N\x86\xbb\xd3'

In [14]:
#Encrypt PS using server's PU key
print('Premaster Secret: ' , PS, "\n")
key= RSA.import_key(open('PU.pem').read())
PU= PKCS1_OAEP.new(key)
ciphertext= PU.encrypt(PS)
print('Encrypted PS: ', ciphertext)

Premaster Secret:  b'\xfb_G\xcc\xe6\x1cy\x04\x1f\x84Xz\x14\xd8]\xf7uX-{\xa6\xc0\x96\x9e*Y\xfd_N\x86\xbb\xd3' 

Encrypted PS:  b'K\xc7\xda\xbdXe\x89\xb0\x17\xcd\x1a\xa18E>\xef\xcf(Iw\xeb\x95\xa7\xc3am\xf8\xf5\xad[\xee6\xe0\xe9\xd0\x05_\xe9\x9c\xc68\n\xb6?\x0cQ\x13_\x1e6\xf2TI$1in/\xb6Av\x9c\xc7\xfc\x0fM\x18\x9em\xd5\xc1S\xd3\xe7\x86\x7f\x1d\xfc\x934\x90\xf9\x1a\xba\xfd\xb5Er\xa7\x8e?\xcda.=\x00\xb0:\xfd\x1d\x02R\x84\xdd\x06\xa7\xd0\x8a\x8b\xc7d\xa7\xa0\x84}\xd8\x8c\x89Z?\x9c\xbd\xa7\x1c\xe9\xb4\x98\x138/\xb5\x9e\xce\xd9B\xe8Y8\xc7\x0f\t\x82>%\xb2\xc0}\x8b*<\xf2\x9e\xce\xb1R\xf2\xfbJ\xe6\x87L{2\xb8JU@5\x91L\xca\xed\x1e\xdf\x12d\xa36\xe8\r\xd6\xa1A\x89\xdd,\xa1\xe4\xc94\x04\xeb\xd76p\xfe\xc3nyP\xda\xc6\xea-\x96\xc3\x1eg\x9e\x89\xf6\x91\x8a\xc2\xa7\x10Q\xe6xF?\xaf\x0c\xb9\x84\xec-\xd6Xq\xfc\x0f\xefg\xeezT\x87\xa8;YCc\xfe\x96~\x08\xbb#n\x976r\xb2g\xca'


In [15]:
#Server decrypts using its PR key
key= RSA.import_key(open('PR.pem').read())
cipher= PKCS1_OAEP.new(key)
plaintext= cipher.decrypt(ciphertext)
print('Decrypted: ', plaintext)

Decrypted:  b'\xfb_G\xcc\xe6\x1cy\x04\x1f\x84Xz\x14\xd8]\xf7uX-{\xa6\xc0\x96\x9e*Y\xfd_N\x86\xbb\xd3'


# Step 6
Pseudorandom number generation

In [16]:
#Generate a subset of the generated random numbers
Rb= client_nonce
Rs= server_nonce
PS_subset= PS[10:17]
Rb_subset= Rb[10:17]
Rs_subset= Rs[10:17]
print(PS_subset, "\n")
print(Rb_subset, "\n")
print(Rs_subset)

b'Xz\x14\xd8]\xf7u' 

b'M\x7f\x93\x04D\t%' 

b'\x1bU7\x9f\xee\xf9\xd1'


In [17]:
#Convert bytes to string
PS_string= ''.join(format(x, '02x') for x in PS_subset)
Rb_string= ''.join(format(x, '02x') for x in Rb_subset)
Rs_string= ''.join(format(x, '02x') for x in Rs_subset)
print(PS_string, "\n")
print(Rb_string, "\n")
print(Rs_string)

587a14d85df775 

4d7f9304440925 

1b55379feef9d1


In [18]:
#Convert string to int for pseudorandom number generator to be able to run properly
PS_int= int(PS_string, 16)
Rb_int= int(Rb_string, 16)
Rs_int= int(Rs_string, 16)
print(PS_int, "\n")
print(Rb_int, "\n")
print(Rs_int)

24904027898509173 

21813842615208229 

7693521765988817


In [19]:
#Pseudorandom number range
num_range= int(PS_string, 16)+ int(Rb_string, 16)+ int(Rs_string, 16)
num_range

54411392279706219

In [20]:
#Generate the 4 communication keys
communication_keys= random.choices(range(num_range), k= 4)
CC= communication_keys[0]
IC= communication_keys[1]
CS= communication_keys[2]
IS= communication_keys[3]
print('CC: ', CC, "\n")
print('IC: ', IC, "\n")
print('CS: ', CS, "\n")
print('IS: ', IS)

CC:  21904689730313800 

IC:  10373079750944994 

CS:  1362028907090671 

IS:  49858509829782056


In [21]:
#Encode the 4 keys into bytes
CC_bytes= str(CC).encode()
IC_bytes= str(IC).encode()
CS_bytes= str(CS).encode()
IS_bytes= str(IS).encode()
print('CC: ', CC_bytes, "\n")
print('IC: ', IC_bytes, "\n")
print('CS: ', CS_bytes, "\n")
print('IS: ', IS_bytes)

CC:  b'21904689730313800' 

IC:  b'10373079750944994' 

CS:  b'1362028907090671' 

IS:  b'49858509829782056'


# Step 7 
Each side will verify the values they received and print either a success or failure message

In [22]:
#Generate client side hash
x= hashlib.sha256()
x.update(PS_subset)
x.update(Rb_subset)
x.update(Rs_subset)
x.update(CC_bytes)
x.update(IC_bytes)
x.digest()
client_hash= x.hexdigest()
client_hash

'100c8238ad7712844a182a4ff75fa1445edfb5fd63274daf536071e72a806b02'

In [23]:
#Server rehashes same values as client
xx= hashlib.sha256()
xx.update(PS_subset)
xx.update(Rb_subset)
xx.update(Rs_subset)
xx.update(CC_bytes)
xx.update(IC_bytes)
xx.digest()
compare= xx.hexdigest()
#compare rehashed values with the client's hash
if client_hash== compare:
    z= 'Matching'
else:
    z= 'Not Matching'
z

'Matching'

In [24]:
#Generate server side hash
s= hashlib.sha256()
s.update(PS_subset)
s.update(Rb_subset)
s.update(Rs_subset)
s.update(CS_bytes)
s.update(IS_bytes)
s.digest()
server_hash= x.hexdigest()
server_hash

'100c8238ad7712844a182a4ff75fa1445edfb5fd63274daf536071e72a806b02'

In [25]:
#Client rehashes same values as server
ss= hashlib.sha256()
ss.update(PS_subset)
ss.update(Rb_subset)
ss.update(Rs_subset)
ss.update(CC_bytes)
ss.update(IC_bytes)
ss.digest()
compare= ss.hexdigest()
#compare rehashed values with the server's hash
if client_hash== compare:
    z= 'Matching'
else:
    z= 'Not Matching'
z

'Matching'

# 2) TLS Simulation using Diffie Hellman

# Step 5
The rest of the steps are the same. 

In [26]:
#a and P are chosen 
if __name__ == '__main__':
    P= 71
    a= 7
    print('Agreed upon Prime Number is: %d'%(P))
    print('Primitive root of Prime number is: %d'%(a), "\n")
     
    #Alice will choose the private key a
    PRa= 2
    print('The Private Key of Alice is: %d'%(PRa))
     
    #gets the generated key
    x= int(pow(a,PRa,P)) 
     
    #Bob will choose the private key b
    PRb= 5
    print('The Private Key of Bob is: %d'%(PRb), "\n")
    
    #gets the generated key
    y= int(pow(a,PRb,P)) 
     
    #Premaster secret key for Alice
    Ska= int(pow(y,PRa,P))
     
    #Premaster secret key for Bob
    Skb= int(pow(x,PRb,P))
     
    print('Shared key of Alice is: %d'%(Ska))
    print('Shared key of Bob is: %d'%(Skb))

Agreed upon Prime Number is: 71
Primitive root of Prime number is: 7 

The Private Key of Alice is: 2
The Private Key of Bob is: 5 

Shared key of Alice is: 45
Shared key of Bob is: 45
